# Import the corresponding Data with Pandas library

In [26]:
import pandas as pd

In [27]:
data = pd.read_csv("../input/simpsons_dataset.csv") 

In [28]:
data.spoken_words=data.spoken_words.str.replace(r"\?",'¿')
data.head()

,raw_character_text,spoken_words
0,Homer Simpson,Never thrown a party¿ What about that big bash...
1,Homer Simpson,"Bart didn't get one vote¿! Oh, this is the wor..."
2,Lisa Simpson,Nothing.
3,Marge Simpson,"Lisa, tell your father."
4,Lisa Simpson,Mr. Bergstrom left today.


In [36]:
data.shape

(54, 2)

In [35]:
#data=data.groupby('raw_character_text').apply(lambda x: ''.join(x.spoken_words))
#len(data[data['autor']=='Lisa Simpson']['texto'].unique())
data[data.raw_character_text=='Bart Simpson']

,raw_character_text,spoken_words
15,Bart Simpson,"Whoa, somebody was bound to say it one day. I ..."
40,Bart Simpson,"Ah, Dad, if just me, Milhouse and Lewis had vo..."
42,Bart Simpson,No.
44,Bart Simpson,Yeah.
46,Bart Simpson,"Well, no."
48,Bart Simpson,"Hey! Thanks, monkey-man."


# API-Chat-Emotions Project

### 0. Importing libraries

In [4]:
import requests

### 1. User endpoints

- (GET) `/user/create/<username>`
  - **Purpose:** Create a user and save into DB
  - **Params:** `username` the user name
  - **Returns:** `user_id`

In [5]:
list_users=data['raw_character_text'].unique()
list_users

array(['Homer Simpson', 'Lisa Simpson', 'Marge Simpson', 'Bart Simpson'],
      dtype=object)

In [6]:
for x in list_users:
    print(requests.get(f"http://localhost:5000/user/create/{x}").text)

User inserted
User inserted
User inserted
User inserted


### 2. Chat endpoints:

- (GET) `/chat/create/<chat_user>`
  - **Purpose:** Create a conversation to load messages
  - **Params:** An array of users ids `[user_id]`
  - **Returns:** `chat_id`

In [37]:
chat_name="Party"
requests.get(f"http://localhost:5000/chat/create/{chat_name}").text

'Chat inserted'

- (GET) `/chat/<chat_id>/adduser/<user_name>`
  - **Purpose:** Add a user to a chat, this is optional just in case you want to add more users to a chat after it's creation.
  - **Params:** `user_id`
  - **Returns:** `chat_id`

In [38]:
list_users=data['raw_character_text'].unique()

In [39]:
chat_id=1
for x in list_users:
    print(requests.get(f"http://localhost:5000/chat/{chat_id}/adduser/{x}").text)

User inserted
User inserted
User inserted
User inserted


- (POST) `/chat/<chat_id>/mensajes/<texto>/adduser/<usuario>`
  - **Purpose:** Add a message to the conversation. Help: Before adding the chat message to the database, check that the incoming user is part of this chat id. If not, raise an exception.
  - **Params:**
    - `chat_id`: Chat to store message
    - `user_id`: the user that writes the message
    - `text`: Message text
  - **Returns:** `message_id`

In [40]:
chat_id=1
for x,y in zip(data['raw_character_text'],data['spoken_words']):
    lista=requests.get(f"http://localhost:5000/chat/{chat_id}/mensajes/{y}/adduser/{x}").text
print(lista)

Users in group, corresponding text inserted


- (GET) `/chat/<chat_id>/list`
  - **Purpose:** Get all messages from `chat_id`
  - **Returns:** json array with all messages from this `chat_id`

In [180]:
numero=1
lista=requests.get(f"http://localhost:5000/chat/{numero}/list").json()[0]['mensajes']
lista[0]

{'autor': 'Homer Simpson',
 'texto': 'Never thrown a party¿ What about that big bash we had with all the champagne and musicians and holy men and everything¿'}

### 3. Sentiment analysis

- (GET) `/chat/<chat_id>/sentiment`
  - **Purpose:** Analyze messages from `chat_id`. Use `NLTK` sentiment analysis package for this task
  - **Returns:** json with all sentiments from messages in the chat
​

In [146]:
chat_id=1
requests.get(f"http://localhost:5000/chat/{chat_id}/sentiment").json()

{'compound': 0.9755, 'neg': 0.073, 'neu': 0.812, 'pos': 0.115}

- Positive sentiment: compound score >= 0.05
- Neutral sentiment: 
    - compound score > -0.05
    - compound score < 0.05
- Negative sentiment: compound score <= -0.05*

### 4. NLP Recommender System

- (GET) `/user/<user_id>/recommend/<chat_id>`
  - **Purpose:** Recommend friend to this user based on chat contents
  - **Returns:** json array with top 3 similar users

In [189]:
user_name="Marge Simpson"
chat_id=1
requests.get(f"http://localhost:5000/user/{user_name}/recommend/{chat_id}").text

'{"Homer Simpson": 0.42663071779135514, "Lisa Simpson": 0.37640402516592697, "Bart Simpson": 0.22551331056809817}'